In [94]:
import pandas as pd
from pandas import DataFrame,Series
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.linear_model import LogisticRegression
from tqdm.auto import tqdm

In [63]:
coupon_area_test = pd.read_csv('./coupon-purchase-prediction/coupon_area_test.csv')
coupon_area_train = pd.read_csv('./coupon-purchase-prediction/coupon_area_train.csv')
coupon_detail_train = pd.read_csv('./coupon-purchase-prediction/coupon_detail_train.csv')
coupon_list_test = pd.read_csv('./coupon-purchase-prediction/coupon_list_test.csv')
coupon_list_train = pd.read_csv('./coupon-purchase-prediction/coupon_list_train.csv')
coupon_visit_train = pd.read_csv('./coupon-purchase-prediction/coupon_visit_train.csv')
prefecture_locations = pd.read_csv('./coupon-purchase-prediction/prefecture_locations.csv')
sample_submission = pd.read_csv('./coupon-purchase-prediction/sample_submission.csv')
user_list = pd.read_csv('./coupon-purchase-prediction/user_list.csv')

In [3]:
coupon_area_test.shape

(2165, 3)

In [4]:
coupon_area_test.head()

,SMALL_AREA_NAME,PREF_NAME,COUPON_ID_hash
0,京都,京都府,c76ea297ebd3a5a4d3bf9f75269f66fa
1,ミナミ他,大阪府,c76ea297ebd3a5a4d3bf9f75269f66fa
2,銀座・新橋・東京・上野,東京都,dd74dc95ca294afa02db40a543ae1763
3,川崎・湘南・箱根他,神奈川県,c65b550cbef918796ad53b1d5b7165c1
4,埼玉,埼玉県,c65b550cbef918796ad53b1d5b7165c1


In [5]:
coupon_area_train.head()

,SMALL_AREA_NAME,PREF_NAME,COUPON_ID_hash
0,埼玉,埼玉県,6b263844241eea98c5a97f1335ea82af
1,千葉,千葉県,cc031f250e8bad1e24060263b9fc0ddd
2,千葉,千葉県,ba5e9b7453ca52ff711635a5d2e8102d
3,千葉,千葉県,3e1ffbedca3569f9e8032d401e8cb4e6
4,千葉,千葉県,782934b6c815b4030ea204eef7d4a734


In [6]:
coupon_detail_train.head()

,ITEM_COUNT,I_DATE,SMALL_AREA_NAME,PURCHASEID_hash,USER_ID_hash,COUPON_ID_hash
0,1,2012-03-28 15:06:06,兵庫,c820a8882374a4e472f0984a8825893f,d9dca3cb44bab12ba313eaa681f663eb,34c48f84026e08355dc3bd19b427f09a
1,1,2011-07-04 23:52:54,銀座・新橋・東京・上野,1b4eb2435421ede98c8931c42e8220ec,560574a339f1b25e57b0221e486907ed,767673b7a777854a92b73b0934ddfae7
2,1,2011-07-16 00:52:49,恵比寿・目黒・品川,36b5f9ba46c44b65587d0b16f2e4c77f,560574a339f1b25e57b0221e486907ed,4f3b5b91d9831192557c056022fdc1f2
3,1,2011-07-16 00:54:53,恵比寿・目黒・品川,2f30f46937cc9004774e576914b2aa1a,560574a339f1b25e57b0221e486907ed,4f3b5b91d9831192557c056022fdc1f2
4,1,2011-07-16 00:55:52,恵比寿・目黒・品川,4d000c64a55ac573d0ae1a8f03677f50,560574a339f1b25e57b0221e486907ed,4f3b5b91d9831192557c056022fdc1f2


In [61]:
coupon_detail_train.columns

Index(['ITEM_COUNT', 'I_DATE', 'SMALL_AREA_NAME', 'PURCHASEID_hash',
       'USER_ID_hash', 'COUPON_ID_hash'],
      dtype='object')

In [62]:
coupon_list_test.head()

AttributeError: 'list' object has no attribute 'head'

In [9]:
coupon_list_test.columns

Index(['CAPSULE_TEXT', 'GENRE_NAME', 'PRICE_RATE', 'CATALOG_PRICE',
       'DISCOUNT_PRICE', 'DISPFROM', 'DISPEND', 'DISPPERIOD', 'VALIDFROM',
       'VALIDEND', 'VALIDPERIOD', 'USABLE_DATE_MON', 'USABLE_DATE_TUE',
       'USABLE_DATE_WED', 'USABLE_DATE_THU', 'USABLE_DATE_FRI',
       'USABLE_DATE_SAT', 'USABLE_DATE_SUN', 'USABLE_DATE_HOLIDAY',
       'USABLE_DATE_BEFORE_HOLIDAY', 'large_area_name', 'ken_name',
       'small_area_name', 'COUPON_ID_hash'],
      dtype='object')

In [10]:
coupon_list_train.head()

,CAPSULE_TEXT,GENRE_NAME,PRICE_RATE,CATALOG_PRICE,DISCOUNT_PRICE,DISPFROM,DISPEND,DISPPERIOD,VALIDFROM,VALIDEND,...,USABLE_DATE_THU,USABLE_DATE_FRI,USABLE_DATE_SAT,USABLE_DATE_SUN,USABLE_DATE_HOLIDAY,USABLE_DATE_BEFORE_HOLIDAY,large_area_name,ken_name,small_area_name,COUPON_ID_hash
0,グルメ,グルメ,50,3000,1500,2011-07-08 12:00:00,2011-07-09 12:00:00,1,2011-07-10,2011-12-08,...,1.0,0.0,0.0,1.0,1.0,0.0,関東,埼玉県,埼玉,6b263844241eea98c5a97f1335ea82af
1,グルメ,グルメ,51,2080,1000,2011-07-01 12:00:00,2011-07-02 12:00:00,1,2011-07-03,2011-12-04,...,1.0,1.0,1.0,1.0,1.0,1.0,関東,千葉県,千葉,cc031f250e8bad1e24060263b9fc0ddd
2,グルメ,グルメ,50,7000,3500,2011-07-12 12:00:00,2011-07-15 12:00:00,3,2011-07-16,2012-01-11,...,1.0,1.0,1.0,1.0,1.0,1.0,関東,千葉県,千葉,ba5e9b7453ca52ff711635a5d2e8102d
3,グルメ,グルメ,50,3000,1500,2011-07-09 12:00:00,2011-07-11 12:00:00,2,2011-07-12,2011-12-01,...,1.0,0.0,0.0,1.0,1.0,1.0,関東,千葉県,千葉,3e1ffbedca3569f9e8032d401e8cb4e6
4,グルメ,グルメ,50,2000,1000,2011-07-05 12:00:00,2011-07-06 12:00:00,1,2011-07-07,2011-12-30,...,1.0,0.0,0.0,1.0,1.0,0.0,関東,千葉県,千葉,782934b6c815b4030ea204eef7d4a734


In [11]:
coupon_visit_train.head()

,PURCHASE_FLG,I_DATE,PAGE_SERIAL,REFERRER_hash,VIEW_COUPON_ID_hash,USER_ID_hash,SESSION_ID_hash,PURCHASEID_hash
0,0,2012-03-28 14:15:00,7,7d3892e54acb559ae36c459978489330,34c48f84026e08355dc3bd19b427f09a,d9dca3cb44bab12ba313eaa681f663eb,673af822615593249e7c6a9a1a6bbb1a,NaN
1,0,2012-03-28 14:17:28,9,7d3892e54acb559ae36c459978489330,34c48f84026e08355dc3bd19b427f09a,d9dca3cb44bab12ba313eaa681f663eb,673af822615593249e7c6a9a1a6bbb1a,NaN
2,0,2012-03-28 14:20:05,16,7d3892e54acb559ae36c459978489330,17c450c3b470c045d35ec22b02daa690,d9dca3cb44bab12ba313eaa681f663eb,673af822615593249e7c6a9a1a6bbb1a,NaN
3,0,2012-03-28 14:23:16,18,7d3892e54acb559ae36c459978489330,91a15e6a95d09e5e01b50747833b317d,d9dca3cb44bab12ba313eaa681f663eb,673af822615593249e7c6a9a1a6bbb1a,NaN
4,0,2012-03-28 14:26:25,20,7d3892e54acb559ae36c459978489330,96fcbc8f6e45d5a2de1661eb140c6e82,d9dca3cb44bab12ba313eaa681f663eb,673af822615593249e7c6a9a1a6bbb1a,NaN


In [12]:
coupon_purchased = coupon_visit_train.PURCHASEID_hash.unique()
len(coupon_purchased)

114775

In [13]:
prefecture_locations.head()

,PREF_NAME,PREFECTUAL_OFFICE,LATITUDE,LONGITUDE
0,北海道,札幌市,43.063968,141.347899
1,青森県,青森市,40.824623,140.740593
2,岩手県,盛岡市,39.703531,141.152667
3,宮城県,仙台市,38.268839,140.872103
4,秋田県,秋田市,39.718600,140.102334


In [14]:
sample_submission.head()

,USER_ID_hash,PURCHASED_COUPONS
0,0000b53e182165208887ba65c079fc21,NaN
1,00035b86e6884589ec8d28fbf2fe7757,NaN
2,0005b1068d5f2b8f2a7c978fcfe1ca06,NaN
3,000cc06982785a19e2a2fdb40b1c9d59,NaN
4,0013518e41c416cd6a181d277dd8ca0b,NaN


In [15]:
user_list.head()

,REG_DATE,SEX_ID,AGE,WITHDRAW_DATE,PREF_NAME,USER_ID_hash
0,2012-03-28 14:14:18,f,25,NaN,NaN,d9dca3cb44bab12ba313eaa681f663eb
1,2011-05-18 00:41:48,f,34,NaN,東京都,560574a339f1b25e57b0221e486907ed
2,2011-06-13 16:36:58,m,41,NaN,愛知県,e66ae91b978b3229f8fd858c80615b73
3,2012-02-08 12:56:15,m,25,NaN,NaN,43fc18f32eafb05713ec02935e2c2825
4,2011-05-22 23:43:56,m,62,NaN,神奈川県,dc6df8aa860f8db0d710ce9d4839840f


In [16]:
user_list.columns

Index(['REG_DATE', 'SEX_ID', 'AGE', 'WITHDRAW_DATE', 'PREF_NAME',
       'USER_ID_hash'],
      dtype='object')

In [17]:
user_feature = user_list.loc[:, ['SEX_ID', 'AGE', 'PREF_NAME','USER_ID_hash']]

In [18]:
user_feature.head

<bound method NDFrame.head of       SEX_ID  AGE PREF_NAME                      USER_ID_hash
0          f   25       NaN  d9dca3cb44bab12ba313eaa681f663eb
1          f   34       東京都  560574a339f1b25e57b0221e486907ed
2          m   41       愛知県  e66ae91b978b3229f8fd858c80615b73
3          m   25       NaN  43fc18f32eafb05713ec02935e2c2825
4          m   62      神奈川県  dc6df8aa860f8db0d710ce9d4839840f
...      ...  ...       ...                               ...
22868      f   24       NaN  2f0a2f36a9f63b6ba2fa3a7e53bef906
22869      m   41       京都府  6ae7811a9c7c58546d6a1567ab098c21
22870      f   35       NaN  a417308c6a79ae0d86976401ec2e3b04
22871      f   59       NaN  4937ec1c86e71d901c4ccc0357cff0b1
22872      f   38       埼玉県  280f0cedda5c4b171ee6245889659571

[22873 rows x 4 columns]>

In [19]:
coupon_feature = coupon_list_train.loc[:, ['PRICE_RATE', 'CATALOG_PRICE',
       'DISCOUNT_PRICE', 'USABLE_DATE_MON', 'USABLE_DATE_TUE',
       'USABLE_DATE_WED', 'USABLE_DATE_THU', 'USABLE_DATE_FRI',
       'USABLE_DATE_SAT', 'USABLE_DATE_SUN', 'USABLE_DATE_HOLIDAY',
       'USABLE_DATE_BEFORE_HOLIDAY', 'COUPON_ID_hash']]

In [20]:
coupon_feature.head()

,PRICE_RATE,CATALOG_PRICE,DISCOUNT_PRICE,USABLE_DATE_MON,USABLE_DATE_TUE,USABLE_DATE_WED,USABLE_DATE_THU,USABLE_DATE_FRI,USABLE_DATE_SAT,USABLE_DATE_SUN,USABLE_DATE_HOLIDAY,USABLE_DATE_BEFORE_HOLIDAY,COUPON_ID_hash
0,50,3000,1500,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,6b263844241eea98c5a97f1335ea82af
1,51,2080,1000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,cc031f250e8bad1e24060263b9fc0ddd
2,50,7000,3500,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,ba5e9b7453ca52ff711635a5d2e8102d
3,50,3000,1500,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,3e1ffbedca3569f9e8032d401e8cb4e6
4,50,2000,1000,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,782934b6c815b4030ea204eef7d4a734


In [21]:
coupon_visit_train.columns

Index(['PURCHASE_FLG', 'I_DATE', 'PAGE_SERIAL', 'REFERRER_hash',
       'VIEW_COUPON_ID_hash', 'USER_ID_hash', 'SESSION_ID_hash',
       'PURCHASEID_hash'],
      dtype='object')

In [22]:
pre_x = user_feature.merge(coupon_visit_train, on = 'USER_ID_hash')

In [23]:
pre_x.columns

Index(['SEX_ID', 'AGE', 'PREF_NAME', 'USER_ID_hash', 'PURCHASE_FLG', 'I_DATE',
       'PAGE_SERIAL', 'REFERRER_hash', 'VIEW_COUPON_ID_hash',
       'SESSION_ID_hash', 'PURCHASEID_hash'],
      dtype='object')

In [24]:
pre_x = coupon_feature.merge(pre_x, left_on = 'COUPON_ID_hash', right_on = 'VIEW_COUPON_ID_hash')

In [47]:
pre_x[['COUPON_ID_hash','VIEW_COUPON_ID_hash']]

,COUPON_ID_hash,VIEW_COUPON_ID_hash
0,6b263844241eea98c5a97f1335ea82af,6b263844241eea98c5a97f1335ea82af
1,6b263844241eea98c5a97f1335ea82af,6b263844241eea98c5a97f1335ea82af
2,6b263844241eea98c5a97f1335ea82af,6b263844241eea98c5a97f1335ea82af
3,6b263844241eea98c5a97f1335ea82af,6b263844241eea98c5a97f1335ea82af
4,6b263844241eea98c5a97f1335ea82af,6b263844241eea98c5a97f1335ea82af
...,...,...
2517201,79303de03e3af7105cded55960d62ac3,79303de03e3af7105cded55960d62ac3
2517202,79303de03e3af7105cded55960d62ac3,79303de03e3af7105cded55960d62ac3
2517203,79303de03e3af7105cded55960d62ac3,79303de03e3af7105cded55960d62ac3
2517204,79303de03e3af7105cded55960d62ac3,79303de03e3af7105cded55960d62ac3


In [26]:
pre_x.columns

Index(['PRICE_RATE', 'CATALOG_PRICE', 'DISCOUNT_PRICE', 'USABLE_DATE_MON',
       'USABLE_DATE_TUE', 'USABLE_DATE_WED', 'USABLE_DATE_THU',
       'USABLE_DATE_FRI', 'USABLE_DATE_SAT', 'USABLE_DATE_SUN',
       'USABLE_DATE_HOLIDAY', 'USABLE_DATE_BEFORE_HOLIDAY', 'COUPON_ID_hash',
       'SEX_ID', 'AGE', 'PREF_NAME', 'USER_ID_hash', 'PURCHASE_FLG', 'I_DATE',
       'PAGE_SERIAL', 'REFERRER_hash', 'VIEW_COUPON_ID_hash',
       'SESSION_ID_hash', 'PURCHASEID_hash'],
      dtype='object')

In [27]:
pre_x.shape

(2517206, 24)

In [28]:
full_data =  pre_x.loc[:, ['PRICE_RATE', 'COUPON_ID_hash',
       'SEX_ID', 'USER_ID_hash', 'PURCHASE_FLG']]

In [29]:
full_data.set_index(['COUPON_ID_hash', 'USER_ID_hash'], inplace = True)

In [30]:
train_x = full_data[['PRICE_RATE', 'SEX_ID']]

In [31]:
train_y = full_data['PURCHASE_FLG']

In [38]:
train_x['SEX_ID'][train_x['SEX_ID']=='m'] = 1
train_x['SEX_ID'][train_x['SEX_ID']=='f'] = 0

<ipython-input-38-c6c422ca7b42>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_x['SEX_ID'][train_x['SEX_ID']=='m'] = 1
/Users/lifangshi/miniconda3/envs/coupon_purchase_prediction/lib/python3.9/site-packages/pandas/core/series.py:992: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(~key, value, inplace=True)
<ipython-input-38-c6c422ca7b42>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_x['SEX_ID'][train_x['SEX_

In [39]:
train_x

PRICE_RATE  \
COUPON_ID_hash                   USER_ID_hash                                   
6b263844241eea98c5a97f1335ea82af 6c3cbc5c3db3c673ffb52cd946dce194          50   
                                 6c3cbc5c3db3c673ffb52cd946dce194          50   
                                 6c3cbc5c3db3c673ffb52cd946dce194          50   
                                 cdeb7bb2ba2fb35aa988a78f2aac1ff1          50   
                                 48363bdca9fdfa03f93b1814414dc69f          50   
...                                                                       ...   
79303de03e3af7105cded55960d62ac3 88bda9e3382ac03e4d0e8ca513dcfe24          60   
                                 9289153e2132f0ad6a78998f0d5ac4f3          60   
                                 c85a5317bb8f72e0baae243c81aa4238          60   
                                 8013c8e81146ad6f1cc99c4227a7c2fd          60   
                                 280f0cedda5c4b171ee6245889659571          60   

                                                                  SEX_ID  
COUPON_ID_hash                   USER_ID_hash                             
6b263844241eea98c5a97f1335ea82af 6c3cbc5c3db3c673ffb52cd946dce194      1  
                                 6c3cbc5c3db3c673ffb52cd946dce194      1  
                                 6c3cbc5c3db3c673ffb52cd946dce194      1  
                                 cdeb7bb2ba2fb35aa988a78f2aac1ff1      0  
                                 48363bdca9fdfa03f93b1814414dc69f      0  
...                                                                  ...  
79303de03e3af7105cded55960d62ac3 88bda9e3382ac03e4d0e8ca513dcfe24      0  
                                 9289153e2132f0ad6a78998f0d5ac4f3      0  
                                 c85a5317bb8f72e0baae243c81aa4238      1  
                                 8013c8e81146ad6f1cc99c4227a7c2fd      1  
                                 280f0cedda5c4b171ee6245889659571      0  

[2517206 rows x 2 columns]

In [100]:
model = LogisticRegression(random_state=0).fit(train_x, train_y)
prediction = model.predict(train_x)

In [46]:
prediction

array([0, 0, 0, ..., 0, 0, 0])

In [43]:
model.predict_proba(train_x)

array([[0.95946523, 0.04053477],
       [0.95946523, 0.04053477],
       [0.95946523, 0.04053477],
       ...,
       [0.94878797, 0.05121203],
       [0.94878797, 0.05121203],
       [0.95370281, 0.04629719]])

In [44]:
model.score(train_x, train_y)

0.9513790289710099

coupon_list_test.columns
Index(['CAPSULE_TEXT', 'GENRE_NAME', 'PRICE_RATE', 'CATALOG_PRICE',
       'DISCOUNT_PRICE', 'DISPFROM', 'DISPEND', 'DISPPERIOD', 'VALIDFROM',
       'VALIDEND', 'VALIDPERIOD', 'USABLE_DATE_MON', 'USABLE_DATE_TUE',
       'USABLE_DATE_WED', 'USABLE_DATE_THU', 'USABLE_DATE_FRI',
       'USABLE_DATE_SAT', 'USABLE_DATE_SUN', 'USABLE_DATE_HOLIDAY',
       'USABLE_DATE_BEFORE_HOLIDAY', 'large_area_name', 'ken_name',
       'small_area_name', 'COUPON_ID_hash'],
      dtype='object')

In [96]:
# test_x = pd.DataFrame(columns=['PRICE_RATE', 'COUPON_ID_hash', 'SEX_ID', 'USER_ID_hash'])
test_x = []
for i in tqdm(coupon_list_test.iterrows(), total=len(coupon_list_test)):
    for j in user_list.iterrows():
        x = pd.concat([i[1][['PRICE_RATE', 'COUPON_ID_hash']],j[1][['SEX_ID', 'USER_ID_hash']]])
#         test_x = test_x.append(x, ignore_index=True)
        test_x.append(x)
test_x = pd.DataFrame(test_x)

  0%|          | 0/310 [00:00<?, ?it/s]

In [97]:
test_x.head()

,PRICE_RATE,COUPON_ID_hash,SEX_ID,USER_ID_hash
0,52,c76ea297ebd3a5a4d3bf9f75269f66fa,f,d9dca3cb44bab12ba313eaa681f663eb
Unnamed 0,52,c76ea297ebd3a5a4d3bf9f75269f66fa,f,560574a339f1b25e57b0221e486907ed
Unnamed 1,52,c76ea297ebd3a5a4d3bf9f75269f66fa,m,e66ae91b978b3229f8fd858c80615b73
Unnamed 2,52,c76ea297ebd3a5a4d3bf9f75269f66fa,m,43fc18f32eafb05713ec02935e2c2825
Unnamed 3,52,c76ea297ebd3a5a4d3bf9f75269f66fa,m,dc6df8aa860f8db0d710ce9d4839840f


In [75]:
test_x

,PRICE_RATE,COUPON_ID_hash,SEX_ID,USER_ID_hash


In [99]:
test_x.set_index(['COUPON_ID_hash', 'USER_ID_hash'], inplace = True)

In [102]:
test_x['SEX_ID'][test_x['SEX_ID']=='m'] = 1
test_x['SEX_ID'][test_x['SEX_ID']=='f'] = 0

<ipython-input-102-9e97ad19b14e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_x['SEX_ID'][test_x['SEX_ID']=='m'] = 1
<ipython-input-102-9e97ad19b14e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_x['SEX_ID'][test_x['SEX_ID']=='f'] = 0


In [108]:
pred = model.predict_proba(test_x)

In [109]:
pred

array([[0.96162709, 0.03837291],
       [0.96162709, 0.03837291],
       [0.957516  , 0.042484  ],
       ...,
       [0.9633949 , 0.0366051 ],
       [0.9633949 , 0.0366051 ],
       [0.9633949 , 0.0366051 ]])

In [110]:
pred[:, 1]

array([0.03837291, 0.03837291, 0.042484  , ..., 0.0366051 , 0.0366051 ,
       0.0366051 ])

In [114]:
sample_submission.head()

,USER_ID_hash,PURCHASED_COUPONS
0,0000b53e182165208887ba65c079fc21,NaN
1,00035b86e6884589ec8d28fbf2fe7757,NaN
2,0005b1068d5f2b8f2a7c978fcfe1ca06,NaN
3,000cc06982785a19e2a2fdb40b1c9d59,NaN
4,0013518e41c416cd6a181d277dd8ca0b,NaN


In [194]:
test_X.head()

,COUPON_ID_hash,USER_ID_hash,PRICE_RATE,SEX_ID
0,c76ea297ebd3a5a4d3bf9f75269f66fa,d9dca3cb44bab12ba313eaa681f663eb,52,0
1,c76ea297ebd3a5a4d3bf9f75269f66fa,560574a339f1b25e57b0221e486907ed,52,0
2,c76ea297ebd3a5a4d3bf9f75269f66fa,e66ae91b978b3229f8fd858c80615b73,52,1
3,c76ea297ebd3a5a4d3bf9f75269f66fa,43fc18f32eafb05713ec02935e2c2825,52,1
4,c76ea297ebd3a5a4d3bf9f75269f66fa,dc6df8aa860f8db0d710ce9d4839840f,52,1


In [252]:
test_X = test_x.reset_index()
for row in tqdm(sample_submission.iterrows(), total = len(sample_submission)):
    idx = pred[test_X['USER_ID_hash'] == row[1]['USER_ID_hash']][:,1].argsort()[-10:]
#     coupon_ids = test_X['COUPON_ID_hash'][test_X['USER_ID_hash'] == row[1]['USER_ID_hash']].reset_index().iloc[idx]['COUPON_ID_hash']
    coupon_ids = coupon_list_test['COUPON_ID_hash'].iloc[idx]
    sample_submission['PURCHASED_COUPONS'][sample_submission['USER_ID_hash'] == row[1]['USER_ID_hash']] = ', '.join(coupon_ids)


  0%|          | 0/22873 [00:00<?, ?it/s]

In [253]:
sample_submission.head()

,USER_ID_hash,PURCHASED_COUPONS
0,0000b53e182165208887ba65c079fc21,"f8cd0c285f925936e2ef808eb3a8296a, b47340cbd4f4..."
1,00035b86e6884589ec8d28fbf2fe7757,"f8cd0c285f925936e2ef808eb3a8296a, b47340cbd4f4..."
2,0005b1068d5f2b8f2a7c978fcfe1ca06,"f8cd0c285f925936e2ef808eb3a8296a, b47340cbd4f4..."
3,000cc06982785a19e2a2fdb40b1c9d59,"f8cd0c285f925936e2ef808eb3a8296a, b47340cbd4f4..."
4,0013518e41c416cd6a181d277dd8ca0b,"f8cd0c285f925936e2ef808eb3a8296a, b47340cbd4f4..."


In [239]:
#', '.join(coupons.reset_index().iloc[idx]['COUPON_ID_hash'])

'f8cd0c285f925936e2ef808eb3a8296a, b47340cbd4f4c57f32637cfb86462cc3, 64b92e53b6e56f7f7bd158ec31887f3d, 16eb5a39ed9efd47cedbfdf8985df147, 9e22b913cb077f8ef844112bcfc72787, 53e6af8b95c14358a345910643a85382, 3ac97c8d840663c600471b44895dc4c0, 784c1314b9f64ae9f3c6e217fb10d15c, 96d275611439e0029db651a914965ea3, c988d799bc7db9254fe865ee6cf2d4ff'

In [254]:
sample_submission.to_csv('result.csv', index=False)

In [113]:
?np.argmax

In [185]:
l1 = np.array([5,2,3,6])
l2 = np.array(['a','a','b','b'])
l1[l2 =='a'] 

array([5, 2])

In [175]:
l2 =='a'

array([ True,  True, False, False])

In [186]:
l1

array([5, 2, 3, 6])

In [178]:
?l1.argsort

In [189]:
np.argsort(l1)

array([1, 2, 0, 3])

In [193]:
l1[l1.argsort()]

array([2, 3, 5, 6])